In [306]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from selenium import webdriver
from pathlib import Path
import time
import random
from itertools import repeat
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import threading
import re
import csv
import itertools
import ast

In [257]:
def resume_iter(iterator,lasIndexProcessed):
    return itertools.islice(iterator,lasIndexProcessed+1,None)

In [2]:
products = pd.read_csv(r'aryaman/csv/categories_and_products/products.csv')

In [338]:
webdriver_path = "..\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=webdriver_path)
# chrome_options = Options()
# chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=cService)

In [94]:
scraped_details = pd.DataFrame({"ASIN":[],"Images":[],"title":[],"productOverview":[],"featureBullets":[],"stars":[],"ratings":[],"listPrice":[],"salePrice":[]})

In [95]:
scraped_details

,ASIN,Images,title,productOverview,featureBullets,stars,ratings,listPrice,salePrice


In [208]:
with open('scraping_details.csv','a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN', 'Images', 'title', 'productOverview', 'featureBullets', 'stars','ratings', 'listPrice', 'salePrice'])
    writer.writeheader()

In [ ]:
products_links = products[['ASIN','Product Link']]
products_links.reset_index()

In [285]:
lastIndexProcessed = 78

In [339]:
with open('scraping_details.csv','a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN', 'Images', 'title', 'productOverview', 'featureBullets', 'stars','ratings', 'listPrice', 'salePrice'])
    for link in resume_iter(products_links.itertuples(),lastIndexProcessed):
        driver.get(link[2])
        time.sleep(1)
        try:
            img_src = driver.find_element(By.CSS_SELECTOR,"#imgTagWrapperId img").get_attribute("src")
            if len(img_src) == 0:
                raise Exception("no img src obtained ",link[0])
            title = driver.find_element(By.CSS_SELECTOR,"#productTitle").text
            if len(title) == 0:
                raise Exception("no title extracted " ,link[0])
            star_rating = float(driver.find_element(By.ID,'acrPopover').get_attribute('title').split(" ")[0])
            if star_rating < 0 or star_rating > 5:
                raise Exception("star rating messed up " ,link[0])
            ratings = driver.find_element(By.ID,'acrCustomerReviewText').text
            ratings = re.sub(r'[^\d]', '',ratings)
            ratings = int(ratings)
            if ratings < 0 or (ratings > 0 and star_rating == 0):
                raise Exception("some ratings messed up at index ",link[0])
            trs = driver.find_element(By.ID,'productOverview_feature_div').find_element(By.TAG_NAME,'table').find_elements(By.TAG_NAME,'tr')
            if len(trs) == 0:
                raise Exception("no table rows of overview found ",link[0])
            overview = {}
            for tr in trs:
                cells = tr.find_elements(By.TAG_NAME,'td')
                if len(cells) == 2:
                    spans_in_first = cells[0].find_elements(By.TAG_NAME,'span')
                    spans_in_second = cells[1].find_elements(By.TAG_NAME,'span')
                    if len(spans_in_first) == len(spans_in_second) == 1:
                        overview[driver.execute_script("return arguments[0].textContent",spans_in_first[0])] = driver.execute_script("return arguments[0].textContent",spans_in_second[0])
            bullets = driver.find_element(By.ID,'feature-bullets').find_element(By.TAG_NAME,'ul').find_elements(By.TAG_NAME,'li')
            if len(bullets) == 0:
                raise Exception("no bullets of about this item found ",link[0])
            bullets_text = list()
            for bullet in bullets:
                span = bullet.find_element(By.TAG_NAME,'span')
                bullets_text.append(driver.execute_script('return arguments[0].textContent',span))
            salePrice = float(re.sub(r'[^\d.]','',driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').find_element(By.CSS_SELECTOR,"span.priceToPay").find_element(By.CSS_SELECTOR,"span.a-price-whole").text))
            if salePrice < 0:
                raise Exception("sale price messed up ",link[0])
            basisPriceElem = driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').find_element(By.CSS_SELECTOR,"span.basisPrice").find_element(By.CSS_SELECTOR,"span.a-text-price").find_element(By.TAG_NAME,'span')
            basisPrice = driver.execute_script("return arguments[0].textContent",basisPriceElem)
            listPrice = float(re.sub(r'[^\d.]', '',basisPrice))
            new_entry = {'ASIN':link[1],'Images':[img_src],'title':title,'productOverview':overview,'featureBullets':bullets_text,'stars':star_rating,'ratings':ratings,'listPrice':listPrice,'salePrice':salePrice}
            writer.writerow(new_entry)
            file.flush()
            lastIndexProcessed = link[0]
            print(title)
        except Exception as e:
            print("Exception occurred while scraping details",e)


NOVEL Baby Wet Wipes (Pack of 6-72 Sheet)
Mylo Baby Gentle Wipes (Pack of 6, 80 Wipes Per Pack) with Organic Coconut & Neem | Anti-Bacterial, pH Balanced, Thick & Soft| With Lid
Mother Sparsh 99% Pure Water Baby Wipes (40X2) Super Saver Travel Friendly Combo Pack | Wipes Made with Plant Based Fabric
Morisons Baby Dreams Soft Baby Wipes With Lid - 72s Pack of 3| Dermatologically tested for baby's gentle skin | Goodness of Aloe Extract & Vitamin E | pH 5.5 | 97.5% Water Wipes | Mositurises & Protects Skin |Alcohol & Paraben Free | 216 Wipes
Mother Sparsh 99% Pure Water Baby Wipes, Pack of 3 (40 X 3 Wipes) | Travel Friendly Pack made with Plant Based Fabric
Mother Sparsh Plant Based Fabric Blue Baby Water Based Wipes (Pack of 4, 80 Pieces)
NOVEL Baby Wet Wipes/Pack With Lid (Pack of 3-80 sheet)
Mee Mee Soft Gentle Baby Wet Wipes (72 Wipes) Pack of 5 | Infused with Aloevera and Lemon Extract | Paraben & Sulfate Free | Cleansing Wipes | Wipes Combo for Babies
Softsens Baby Gentle Cloth Wipe

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.92)
Stacktrace:
	GetHandleVerifier [0x00007FF6FBB13AB5+28005]
	(No symbol) [0x00007FF6FBA783B0]
	(No symbol) [0x00007FF6FB91580A]
	(No symbol) [0x00007FF6FB9025BC]
	(No symbol) [0x00007FF6FB902480]
	(No symbol) [0x00007FF6FB918181]
	(No symbol) [0x00007FF6FB9AB1D9]
	(No symbol) [0x00007FF6FB98B7E3]
	(No symbol) [0x00007FF6FB9575C8]
	(No symbol) [0x00007FF6FB958731]
	GetHandleVerifier [0x00007FF6FBE0643D+3118829]
	GetHandleVerifier [0x00007FF6FBE56C90+3448640]
	GetHandleVerifier [0x00007FF6FBE4CF0D+3408317]
	GetHandleVerifier [0x00007FF6FBBDA40B+841403]
	(No symbol) [0x00007FF6FBA8340F]
	(No symbol) [0x00007FF6FBA7F484]
	(No symbol) [0x00007FF6FBA7F61D]
	(No symbol) [0x00007FF6FBA6EB79]
	BaseThreadInitThunk [0x00007FF8EFBB257D+29]
	RtlUserThreadStart [0x00007FF8F154AF08+40]


In [343]:
lastIndexProcessed

782

In [345]:
products_links.index

RangeIndex(start=0, stop=7092, step=1)

In [344]:
products_links[products_links['ASIN']=='B006NVDWGE']

,ASIN,Product Link
782,B006NVDWGE,https://www.amazon.in/dp/B006NVDWGE


In [346]:
products_links.iloc[760:790]

,ASIN,Product Link
760,B08Z37LFZB,https://www.amazon.in/dp/B08Z37LFZB
761,B0CRRTQJXR,https://www.amazon.in/dp/B0CRRTQJXR
762,B0CTTHFZ1S,https://www.amazon.in/dp/B0CTTHFZ1S
763,B0CM6K7MSB,https://www.amazon.in/dp/B0CM6K7MSB
764,B0CW9NPPCC,https://www.amazon.in/dp/B0CW9NPPCC
765,B08FTQXWC7,https://www.amazon.in/dp/B08FTQXWC7
766,B0BCSZL5SF,https://www.amazon.in/dp/B0BCSZL5SF
767,B07CJPX7KW,https://www.amazon.in/dp/B07CJPX7KW
768,B01CCGW4OE,https://www.amazon.in/dp/B01CCGW4OE
769,B0CH8SRX8M,https://www.amazon.in/dp/B0CH8SRX8M


In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ASIN             87 non-null     object 
 1   Images           87 non-null     object 
 2   title            87 non-null     object 
 3   productOverview  87 non-null     object 
 4   featureBullets   87 non-null     object 
 5   stars            87 non-null     float64
 6   ratings          87 non-null     int64  
 7   listPrice        87 non-null     float64
 8   salePrice        87 non-null     float64
dtypes: float64(3), int64(1), object(5)
memory usage: 6.2+ KB


In [347]:
df = pd.read_csv('scraping_details.csv',encoding='utf-8')

In [370]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ASIN             463 non-null    object 
 1   Images           463 non-null    object 
 2   title            463 non-null    object 
 3   productOverview  463 non-null    object 
 4   featureBullets   463 non-null    object 
 5   stars            463 non-null    float64
 6   ratings          463 non-null    int64  
 7   listPrice        463 non-null    float64
 8   salePrice        463 non-null    float64
dtypes: float64(3), int64(1), object(5)
memory usage: 32.7+ KB


In [371]:
emer1 = df['ASIN'].tolist()

In [352]:
emer2 = products_links['ASIN'].tolist()

In [365]:
for i in range(434,783):
    if emer1[i-32] != emer2[i]:
        print(i)

434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683


In [391]:
indexes_fethced = list()
for ulo in emer1:
    indexes_fethced.append(int(products_links[products_links['ASIN']==ulo].index[0]))

In [392]:
indexes_fethced.sort()

In [394]:
len(indexes_fethced)

463

In [399]:
boolpool = [False]*783
for i in indexes_fethced:
    boolpool[i] = True
for x in range(0,len(boolpool)):
    if boolpool[x] == False:
        print(x)

114
115
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
186
191
193
194
211
217
218
219
220
236
237
241
248
259
265
275
281
305
312
314
315
316
317
318
319
320
321
322
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
343
344
345
347
348
349
351
352
354
355
356
357
358
359
360
361
363
364
365
367
369
370
371
372
375
376
378
379
380
381
382
383
394
397
405
410
417
419
423
426
436
440
441
442
445
447
448
463
465
466
469
479
480
481
487
488
489
490
491
492
493
494
495
496
498
499
501
502
503
504
506
508
509
513
514
515
516
518
520
522
524
525
526
527
528
529
530
531
534
535
536
537
538
540
542
543
545
546
547
549
550
553
555
556
558
559
560
561
562
563
565
566
567
568
569
572
573
575
577
578
579
580
581
584
585
587
589
590
591
593
594
595
597
598


In [398]:
boolpool.__len__()

783

In [359]:
haw = list(products_links.itertuples())

In [361]:
haw[114]

Pandas(Index=114, ASIN='B0CS3QBX7K', _2='https://www.amazon.in/dp/B0CS3QBX7K')

In [366]:
products_links[products_links['ASIN']=='B0DFH93LP9']

,ASIN,Product Link
116,B0DFH93LP9,https://www.amazon.in/dp/B0DFH93LP9


In [367]:
products_links.iloc[115]

ASIN                                     B0DGTRFH9B
Product Link    https://www.amazon.in/dp/B0DGTRFH9B
Name: 115, dtype: object

In [356]:
df.iloc[114]

ASIN                                                      B0DFH93LP9
Images             ['https://m.media-amazon.com/images/I/615onOV0...
title              Johnson's Baby Wet Wipes | Alcohol & Soap Free...
productOverview    {'Brand': "Johnson's", 'Skin Type': 'All', 'Ma...
featureBullets     [" Johnson's baby wipes with lid, protects new...
stars                                                            4.6
ratings                                                            3
listPrice                                                     1300.0
salePrice                                                      648.0
Name: 114, dtype: object

In [369]:
products_links['ASIN'] == 'B075GHP2LX'

0        True
1       False
2       False
3       False
4       False
        ...  
7087    False
7088    False
7089    False
7090    False
7091    False
Name: ASIN, Length: 7092, dtype: bool

In [329]:
df['Images'] = df['Images'].apply(lambda x :ast.literal_eval(x))
df['productOverview'] = df['productOverview'].apply(lambda x :ast.literal_eval(x))
df['featureBullets'] = df['featureBullets'].apply(lambda x :ast.literal_eval(x))

In [332]:
sampledf = df['featureBullets'].sample(6)
for i in sampledf:
    print(i[0])

 New & Improved Huggies Complete Comfort Wonder Pants - India's Fastest Absorbing diaper with Patented Dry Xpert Channel; Small size / S Size baby diapers  
 Our No.1 soft Pants: New & Premium Huggies Natural Soft comes with India's 1st Cloud Touch BeltTM with cloud softness all over to help prevent red marks; Large (L) Size (9-14 Kgs), Sumo Pack of 50x3 diapers  
 12-HOURS OF ABSORPTION!  
 Choosing the right diaper size based on baby weight is important. The diaper capacity may vary among babies depending on the volume of pee and posture  
 Ideal for gently cleansing and moisturizing your baby's delicate skin during diaper change or for cleaning your baby's face and hands during mealtime, playtime and while travelling  
 LEADING BABYCARE BRAND: We know your babies require utmost care & protection while they grow. Your tot's skin is as soft as cotton and too sensitive to handle. Thus, it needs special care & attention every single moment.  
